In [5]:
import openai
import json

prompt = "桌上有 2 个苹果，四个桃子和 3 本书，一共有几个水果？"
messages = [
    {"role": "system", "content": "你是一个小学数学老师，你要教学生加法"},
    {"role": "user", "content": prompt}
]

tools = [{                            # 用 JSON 描述函数。可以定义多个，但是只有一个会被调用，也可能都不会被调用
    "type": "function",   
    "function": {
        "name": "zlwm",
        "parameters": {
            "type": "object",
            "properties": {
                "numbers": {
                    "type": "array",
                    "items": {
                            "type": "number"
                    },
                },
            },
        },
    },
}]

def get_completion(messages, model="gpt-3.5-turbo-1106"):
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0,          # 模型输出的随机性，0 表示随机性最小
        tools = tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
    )
    
    return response.choices[0].message
response_message = get_completion(messages)
messages.append(response_message)  # 把大模型的回复加入到对话中

In [10]:
print(response_message)
print(messages)

def zlwm():
        return json.dumps({"jieguo":555})


ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_zlvYoOahbSRq3NAyHCNoq0H6', function=Function(arguments='{"numbers":[2,4]}', name='zlwm'), type='function')])
[{'role': 'system', 'content': '你是一个小学数学老师，你要教学生加法'}, {'role': 'user', 'content': '桌上有 2 个苹果，四个桃子和 3 本书，一共有几个水果？'}, ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_zlvYoOahbSRq3NAyHCNoq0H6', function=Function(arguments='{"numbers":[2,4]}', name='zlwm'), type='function')])]


In [9]:
tool_calls = response_message.tool_calls

if tool_calls:
    available_functions = {
            "zlwm": zlwm,
    }
    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_to_call = available_functions[function_name]
        function_args = json.loads(tool_call.function.arguments)     #{'numbers': [2, 4]}
        function_response = function_to_call()
            jieguo = function_args.get("location"),
        messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
        )  # extend conversation with function response

        second_response = openai.chat.completions.create(
            model="gpt-3.5-turbo-1106",
            messages=messages,
        )  # get a new response from the model where it can see the function response
        print(second_response)   

TypeError: Object of type set is not JSON serializable

In [28]:
print(type(response))
print(response)
print(type(response_message))
print(response_message)

<class 'openai.types.chat.chat_completion_message.ChatCompletionMessage'>
ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_S8UWk5rlSC7qCmcyWCb6iobX', function=Function(arguments='{"numbers":[2,4]}', name='sum'), type='function')])
<class 'openai.types.chat.chat_completion_message.ChatCompletionMessage'>
ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_FQr9t1zFTBDYu6y8UqwNEMHb', function=Function(arguments='{"numbers":[2,4]}', name='sum'), type='function')])
